Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from scipy.stats import multivariate_normal
from scipy import stats

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

*Посмотрим на наши данные визуально*

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


*Оценим наши данные на пропуски*

In [5]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

*Проверим данные на дубликаты*

In [6]:
data.duplicated().sum()

153

*Удалим дубликаты*

In [7]:
data.drop_duplicates(inplace=True)

*Посмотрим на целевую переменную*

In [8]:
data['Страховые выплаты'].value_counts()

0    4284
1     423
2     114
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

*Видим, что целевая переменная у нас количественная, дискретная*

### Вывод

*Мы провели легкий исследовательский анализ наших данных, нашли некоторое количество дубликатов, в остальном наши данные обработаны и пригодны для построения модели машинного обучения.*

## 2. Умножение матриц

Попробуем доказать/опровергнуть утверждение: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:**

Мы знаем, что 
$$
X = X*E
$$
$$
E = P*P^{-1}
$$
Выразив одно утверждение через другое, мы имеем
$$
X^{-1} = X*P
$$
Из этих правил, мы можем сделать вывод, что:
$$
P^{-1} * w = w^{-1}
$$
Теперь подставим данные утверждения в формулу расчета MSE, мы имеем:
$$
MSE(Xw, y) = MSE(XEw, y) = MSE(XPP^{-1}w, y) = MSE(X^{-1}w^{-1}, y) = MSE((Xw)^{-1}, y)
$$
Данное утверждение справедиво, если пространство признаков имеет размерность m x n, а матрица, на которую умножаем имеет размер n x n и, по определению, обратима.

Докажем данное утверждение и для формулы весов (домножим на P):

$$(X^T X)^{-1} X^T y = ((XP)^T (XP))^{-1} (XP)^T y = ((X^{-1})^T (X^{-1}))^{-1} (X^{-1})^T y = (X^T X) (X^T)^{-1} y = w^{-1}$$

Мы видим, что выражение $(X^T X) (X^{-1})^T y$, обратно пространству признаков при умножении на P, используя равенство $P^{-1} * w = w^{-1}$, мы можем сделать вывод об эквивалентности данных

Из вышестоящего выражения, мы доказали, что $w = P * w^{-1}$, умножим $a$ на $P$, получим $a = Xw = XP^{-1}w$.

Подставив $w = P * w^{-1}$, получим $a = Xw = XP^{-1} * P * w^{-1} = XEw = Xw$

По формуле обучения, вектора весов, обученных на признаках $X$ и признаках $XP$ (где P-обратимая матрица), выглядят так:

$$
w = (X^T X)^{-1} X^T y
$$

$$
u = ((X P)^T X P)^{-1} (X P)^T y
$$
Воспользовавшись свойствами:

(1) $(A B)^T = B^T A^T$

(2) $(A B)^{-1} = B^{-1} A^{-1}$ для квадратных обратимых матриц $A$ и $B$

(3) $A^{-1} A = E$

(4) $E A = А E = A$

Преобразуем выражение $u = ((X P)^T X P)^{-1} (X P)^T y$ = $((X^{-1} * (P^{-1})^T (X^{-1} * P^{-1}))^{-1} (X^{-1} * P^{-1})^T y$, раскроем скобки, воспользовавшись вышеописанными свойствами и сделая перестановки, мы получим:

$u = P^{-1} * (X^T X)^{-1} X^T y = P^{-1} w$

Далее умножим пространство предсказаний на Р: $a_{xp} = X P u = X P P^{-1} w = X w$

## 3. Алгоритм преобразования

**Алгоритм**

Для кодирования обратимой матрицей, мы умножаем пространство признаков на обратимую матрицу из многомерного нормального распределения, данным действием мы сдвигаем пространство признаков в том же пространстве, что не нарушает структуру и при вычислении MSE не влияет никак на качество модели, так результирующее пространство предсказаний остается неизменным и, соответственно, разница между $y1 - y2$, остается неизменным.

Зная кодирующую матрицу, мы довольно надежно можем деперсонализировать пространство признаков, которое будет проблемно расшифровать, не зная декодирующую матрицу, но пространство признаков можно преобразовать к первоначальному виду, умножив на обратную матрицу, что и будет продемонстрировано дальше.

## 4. Проверка алгоритма

*Разделим наши данные на признаки и целевую переменную*

In [9]:
X_original = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']

*Обучим модель линейной регрессии на не преобразованных данных* 

In [10]:
model = LinearRegression()
score_before = cross_val_score(model, X_original, y, scoring='r2', cv=5).mean()
score_dict = {'Точность модели до преобразования': score_before}; score_dict

{'Точность модели до преобразования': 0.42779425802804916}

***

*Теперь проведем кодирование признаков, в соответствии с алгоритмом, а именно умножим пространство признаков на обратимую матрицу из пространства нормального распределения*

In [12]:
P = np.random.normal(size=(4, 4))

*Убедимся, что наша матрица обратима, умножим на обратную, должна получится еденичная матрица*

In [13]:
P @ np.linalg.inv(P)

array([[ 1.00000000e+00, -1.41846309e-16,  2.30055534e-16,
         8.53359183e-17],
       [ 2.74642151e-17,  1.00000000e+00, -1.52212126e-16,
         3.22415879e-17],
       [-9.72661429e-17,  9.19305072e-17,  1.00000000e+00,
        -9.23195349e-17],
       [-1.19260614e-18,  5.68217254e-17,  1.15497326e-16,
         1.00000000e+00]])

*Видим, что действительно получилась еденичная матрица, за исключением погрешности матричных вычислений в 15 знаке*

In [14]:
#Проведем кодирование пространства признаков
X = X_original @ P

*Посмотрим визуально на признаки*

In [15]:
X.head()

,0,1,2,3
0,-15468.025668,-42294.373289,24346.975716,-30911.000394
1,-11847.875707,-32379.049312,18665.189071,-23676.769607
2,-6546.626176,-17888.227341,10317.482047,-13083.515622
3,-13008.629114,-35576.504206,20460.506732,-25994.171317
4,-8137.309555,-22247.038664,12815.636731,-16262.390674


*Отлично, набор случайных цифр*

*Обучим модель линейно регресси на преобразованных данных и сравним качество модели*

In [16]:
score_after = cross_val_score(model, X, y, scoring='r2', cv=5).mean()
score_dict['Точность модели после преобразования'] = score_after
score_dict['Разница оценки качества'] = score_after - score_before; score_dict

{'Точность модели до преобразования': 0.42779425802804916,
 'Точность модели после преобразования': 0.42779425802807136,
 'Разница оценки качества': 2.220446049250313e-14}

*Предположим нулевую гипотезу - качество не изменилось*

In [17]:
alpha = .05
result = stats.ttest_ind(X_original, X)
if result.pvalue.mean() > alpha:
    print("Пространство признаков изменилось")
else:
    print("Пространство признаков не изменилось")

Пространство признаков не изменилось


*Мы видим, что качество модели практически не изменилось, разница есть на 13 знаке после запятой, что подтверждает и статистический Т-тест*

***

*Попробуем преобразовать наши данные в первоначальный вид*

In [18]:
X = X @ np.linalg.inv(P); X.head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-3.477453e-12,46.0,38000.0,1.000000e+00
2,-1.764261e-12,29.0,21000.0,-1.709641e-12
3,-4.637219e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-3.231739e-12


*Видим, что данные восстановленны, практически 100%, имеется артефакты преобразований на уровне 11-14 знак после десятичного разделителя.*

*Сравним ее с изначальной*

In [19]:
pd.DataFrame(X.values - X_original.values)

,0,1,2,3
0,-4.036993e-12,4.320100e-12,-7.275958e-12,-8.114731e-12
1,-3.477453e-12,1.932676e-12,0.000000e+00,-4.024558e-12
2,-1.764261e-12,2.739142e-12,3.637979e-12,-1.709641e-12
3,-4.637219e-12,5.190515e-12,0.000000e+00,-3.175682e-12
4,-1.587397e-12,2.906120e-12,0.000000e+00,-3.231739e-12
...,...,...,...,...
4842,-2.932608e-12,2.131628e-12,0.000000e+00,-4.026557e-12
4843,-4.545450e-12,4.135359e-12,0.000000e+00,-4.592327e-12
4844,-3.903995e-12,2.049916e-12,0.000000e+00,-2.695177e-12
4845,-2.385536e-12,1.509903e-12,3.637979e-12,-2.893685e-12


*Посмотрим на describe*

In [20]:
print(X.describe()); print(X_original.describe())

                  0            1             2             3
count  4.847000e+03  4847.000000   4847.000000  4.847000e+03
mean   4.984527e-01    31.023932  39895.811842  1.203425e+00
std    5.000492e-01     8.487995   9972.953985  1.098664e+00
min   -9.486570e-12    18.000000   5300.000000 -1.609578e-11
25%   -3.503179e-12    24.000000  33200.000000 -2.534875e-12
50%   -8.240155e-13    30.000000  40200.000000  1.000000e+00
75%    1.000000e+00    37.000000  46600.000000  2.000000e+00
max    1.000000e+00    65.000000  79000.000000  6.000000e+00
               Пол      Возраст      Зарплата  Члены семьи
count  4847.000000  4847.000000   4847.000000  4847.000000
mean      0.498453    31.023932  39895.811842     1.203425
std       0.500049     8.487995   9972.953985     1.098664
min       0.000000    18.000000   5300.000000     0.000000
25%       0.000000    24.000000  33200.000000     0.000000
50%       0.000000    30.000000  40200.000000     1.000000
75%       1.000000    37.000000  46600

*Видим, что получилась нулевая матрица, с погрешностями от матричных вычислений*

### Вывод

*Мы провели реализацию алгоритма кодирования признаков и убедились, что данный алгоритм никак не влияет на качество модели - данный инструмент можно использовать в дальнейшем для кодирования признаков.*